In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

from metpy.calc import dewpoint_from_relative_humidity
from metpy.units import units

import sys
sys.path.append('../../') # lets us import ptype package from the subdir

import sounding_utils
from sounding_utils import open_ds_dkimpara, filter_latlon
from importlib import reload

from functools import partial
import cartopy.crs as ccrs

#import ptype.

## Select forecast hour:

In [2]:
hour = '0000'

In [3]:
model='rap'
rap = open_ds_dkimpara(hour,model)

In [4]:
model='hrrr'
hrrr = open_ds_dkimpara(hour,model)

In [5]:
model='gfs'
gfs = open_ds_dkimpara(hour,model)


In [6]:
datasets = [gfs, hrrr, rap]

In [7]:
bbox = {"lon_min": max([ds.longitude.min() for ds in datasets]),
        "lon_max": min([ds.longitude.max() for ds in datasets]),
        "lat_min": max([ds.latitude.min() for ds in datasets]),
        "lat_max": min([ds.latitude.max() for ds in datasets])
       }

In [9]:
from joblib import dump, load

In [10]:
dump(bbox, 'bbox_dict')

['bbox_dict']

In [11]:
bload = load('bbox_dict')
print(bload)

{'lon_min': <xarray.DataArray 'longitude' ()>
dask.array<_nanmin_skip-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>, 'lon_max': <xarray.DataArray 'longitude' ()>
dask.array<_nanmax_skip-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>, 'lat_min': <xarray.DataArray 'latitude' ()>
dask.array<_nanmin_skip-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>, 'lat_max': <xarray.DataArray 'latitude' ()>
dask.array<_nanmax_skip-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>}


In [12]:
[b.values for k,b in bload.items()]

[array(225.90453, dtype=float32),
 array(299.0828, dtype=float32),
 array(21.138123, dtype=float32),
 array(52.615654, dtype=float32)]

In [27]:
def filter(ds):
    bbox = load('bbox_dict')
    
    mask = (
            (ds.latitude <= bbox['lat_max']) &
            (ds.latitude >= bbox['lat_min']) &
            (ds.longitude <= bbox['lon_max']) &
            (ds.longitude >= bbox['lon_min'])
           )
    
    return ds.where(mask.compute(), drop=True)


In [28]:
filtered = [filter(ds) for ds in datasets]